In [1]:
# dependency set up csv file as dataframe

import pandas as pd
path = "resources/purchase_data.csv"
purchase_df = pd.read_csv(path)
purchase_df.head()

Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price  
0  Extraction, Quickblade Of Trembling Hands   3.53  
1                          Frenzied Scimitar   1.56  
2                               Final Critic   4.88  
3                                Blindscythe   3.27  
4                                       Fury   1.44

In [2]:
    # Player Count
# display total number of players
players_count = len(purchase_df["SN"].unique())
print(f'Total Players: {players_count}')

Total Players: 576


In [3]:
    # Purchasing Analysis (Total)
# finding the needed values
items_count = len(purchase_df["Item Name"].unique())
av_price = purchase_df["Price"].mean()
total_purchase_count = len(purchase_df["Price"])
total_revenue = purchase_df["Price"].sum()

# put the data into new dataframe, and change the money related to right format
purchase_adf = pd.DataFrame({"Number of Unique Items": [items_count],
                             "Average Purchase Price": [av_price],
                             "Total Number of Purchases": [total_purchase_count],
                             "Total Revenue": [total_revenue]})
purchase_adf = purchase_adf.style.format({"Average Purchase Price": "${:.2f}",
                                           "Total Revenue": "${:,.2f}"})
purchase_adf

In [4]:
    # Gender Demographics
# checking the gender list
gender_list = purchase_df["Gender"].unique()
gender_list

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

In [5]:
    #Gender Demographics
# get separate dataframe for each gender found from list above
# this is needed for next part as well
male_df = purchase_df.loc[purchase_df["Gender"] == "Male", :]
other_df = purchase_df.loc[purchase_df["Gender"] == "Other / Non-Disclosed", :]
female_df = purchase_df.loc[purchase_df["Gender"] == "Female", :]

# get count and percentage for each gender
male_count = len(male_df["SN"].unique())
other_count = len(other_df["SN"].unique())
female_count = len(female_df["SN"].unique())

male_percent = (male_count / players_count) * 100
other_percent = (other_count / players_count) * 100
female_percent = (female_count / players_count) * 100

# create dataframe for analysis with right format
gender_adf = pd.DataFrame({"Gender": gender_list,
                           "Total Count": [male_count, other_count, female_count],
                           "Percentage": [male_percent, other_percent, female_percent]})
gender_adf = gender_adf.sort_values("Total Count", ascending=False)
gender_adf = gender_adf.set_index("Gender")
gender_adf["Percentage"] = gender_adf["Percentage"].map("{:.2f}%".format)
gender_adf

Total Count Percentage
Gender                                       
Male                           484     84.03%
Female                          81     14.06%
Other / Non-Disclosed           11      1.91%

In [6]:
    # Purchasing Analysis (Gender)
# make gender purchase count dataframe
gender_purchase_df = purchase_df["Gender"].value_counts().to_frame()

# get the average purchase values and add to the dataframe
male_mean = male_df["Price"].mean()
female_mean = female_df["Price"].mean()
other_mean = other_df["Price"].mean()

gender_mean_list = [male_mean, female_mean, other_mean]
gender_purchase_df["Average Purchase Price"] = gender_mean_list

# get total purchase values and add to the dataframe
male_total = male_df["Price"].sum()
female_total = female_df["Price"].sum()
other_total = other_df["Price"].sum()

gender_total_list = [male_total, female_total, other_total]
gender_purchase_df["Total Purchase Value"] = gender_total_list

# get average total purchase per person and add to the dataframe
male_pp = male_total/male_count
female_pp = female_total/female_count
other_pp = other_total/other_count

gender_pp_list = [male_pp, female_pp, other_pp]
gender_purchase_df["Average Total Purchase per Person"] = gender_pp_list

# make the dataframe cleaner with number values
gender_purchase_df = gender_purchase_df.rename(columns={"Gender": "Purchase Count"})
gender_purchase_df["Average Purchase Price"] = gender_purchase_df["Average Purchase Price"].map("${:,.2f}".format)
gender_purchase_df["Total Purchase Value"] = gender_purchase_df["Total Purchase Value"].map("${:,.2f}".format)
gender_purchase_df["Average Total Purchase per Person"] = gender_purchase_df["Average Total Purchase per Person"].map("${:,.2f}".format)
gender_purchase_df

Purchase Count Average Purchase Price  \
Male                              652                  $3.02   
Female                            113                  $3.20   
Other / Non-Disclosed              15                  $3.35   

                      Total Purchase Value Average Total Purchase per Person  
Male                             $1,967.64                             $4.07  
Female                             $361.94                             $4.47  
Other / Non-Disclosed               $50.19                             $4.56

In [7]:
    # Age Demographics
# establish bins for ages
agebin = [0, 9, 14, 19, 24, 29, 34, 39, 100]
agelabel = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# categorize the existing players using the age bins.
purchase_df["Age Group"] = pd.cut(purchase_df["Age"], agebin, labels=agelabel)

# create new dataframe with just counting unique players per age group
agegroup_df = purchase_df.groupby("Age Group")["SN"].nunique().to_frame()
agegroup_df = agegroup_df.rename(columns={"SN": "Total Count"})

# calculate the percentage of players
agegroup_df["Percentage of Players"] = (agegroup_df["Total Count"]/players_count) * 100

# make the dataframe cleaner with number values
agegroup_df["Percentage of Players"] = agegroup_df["Percentage of Players"].map("{:.2f}%".format)
agegroup_df

Total Count Percentage of Players
Age Group                                   
<10                 17                 2.95%
10-14               22                 3.82%
15-19              107                18.58%
20-24              258                44.79%
25-29               77                13.37%
30-34               52                 9.03%
35-39               31                 5.38%
40+                 12                 2.08%

In [8]:
    # Purchasing Analysis (Age)
# get the purchase count per age group using the cut dataframe earlier and clean
agegroup_purchase_df = purchase_df.groupby("Age Group")
agegroup_purchase_adf = agegroup_purchase_df["SN"].count().to_frame()
agegroup_purchase_adf = agegroup_purchase_adf.rename(columns={"SN": "Purchase Count"})

# get the average purchase price, total purchase value, and average total per person
agegroup_purchase_adf["Average Purchase Price"] = agegroup_purchase_df["Price"].mean()
agegroup_purchase_adf["Total Purchase Value"] = agegroup_purchase_df["Price"].sum()
agegroup_purchase_adf["Average Total Purchase per Person"] = agegroup_purchase_adf["Total Purchase Value"]/agegroup_df["Total Count"]

# map the numerical values
agegroup_purchase_adf["Average Purchase Price"] = agegroup_purchase_adf["Average Purchase Price"].map("${:.2f}".format)
agegroup_purchase_adf["Total Purchase Value"] = agegroup_purchase_adf["Total Purchase Value"].map("${:.2f}".format)
agegroup_purchase_adf["Average Total Purchase per Person"] = agegroup_purchase_adf["Average Total Purchase per Person"].map("${:.2f}".format)
agegroup_purchase_adf

Purchase Count Average Purchase Price Total Purchase Value  \
Age Group                                                               
<10                    23                  $3.35               $77.13   
10-14                  28                  $2.96               $82.78   
15-19                 136                  $3.04              $412.89   
20-24                 365                  $3.05             $1114.06   
25-29                 101                  $2.90              $293.00   
30-34                  73                  $2.93              $214.00   
35-39                  41                  $3.60              $147.67   
40+                    13                  $2.94               $38.24   

          Average Total Purchase per Person  
Age Group                                    
<10                                   $4.54  
10-14                                 $3.76  
15-19                                 $3.86  
20-24                                 $4.32  
25-29                                 $3.81  
30-34                                 $4.12  
35-39                                 $4.76  
40+                                   $3.19

In [9]:
    # Top Spenders
# group by the account names and make new dataframe to display
sn_df = purchase_df.groupby("SN")
sn_adf = sn_df["Purchase ID"].count().to_frame()
sn_adf = sn_adf.rename(columns={"Purchase ID": "Purchase Count"})

# get the average purchase price and total purchase value
sn_adf["Average Purchase Price"] = sn_df["Price"].mean()
sn_adf["Total Purchase Value"] = sn_df["Price"].sum()

# organize so its in descending order of total purchase value
sn_adf = sn_adf.sort_values(by=["Total Purchase Value"], ascending=False)

# reformat the dataframe
sn_adf["Average Purchase Price"] = sn_adf["Average Purchase Price"].map("${:.2f}".format)
sn_adf["Total Purchase Value"] = sn_adf["Total Purchase Value"].map("${:.2f}".format)
sn_adf

Purchase Count Average Purchase Price Total Purchase Value
SN                                                                     
Lisosia93                 5                  $3.79               $18.96
Idastidru52               4                  $3.86               $15.45
Chamjask73                3                  $4.61               $13.83
Iral74                    4                  $3.40               $13.62
Iskadarya95               3                  $4.37               $13.10
...                     ...                    ...                  ...
Ililsasya43               1                  $1.02                $1.02
Irilis75                  1                  $1.02                $1.02
Aidai61                   1                  $1.01                $1.01
Chanirra79                1                  $1.01                $1.01
Alo38                     1                  $1.00                $1.00

[576 rows x 3 columns]

In [10]:
    # Most Popular Items
# retrieve the item id, name, and price columns and make new dataframe for display later
item_df = purchase_df[["Item ID", "Item Name", "Price"]]
item_df = item_df.groupby(["Item ID", "Item Name"])
popular_df = item_df.count()
popular_df = popular_df.rename(columns={"Price": "Purchase Count"})

# get the price and total of each item
popular_df["Item Price"] = item_df["Price"].mean()
popular_df["Total Purchase Value"] = item_df["Price"].sum()

# reorganize in descending order of purchase counts
popular_df = popular_df.sort_values(by=["Purchase Count"], ascending=False)

# change the numbering format
popular_df["Item Price"] = popular_df["Item Price"].map("${:.2f}".format)
popular_df["Total Purchase Value"] = popular_df["Total Purchase Value"].map("${:.2f}".format)
popular_df

Purchase Count  \
Item ID Item Name                                                      
92      Final Critic                                              13   
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
132     Persuasion                                                 9   
108     Extraction, Quickblade Of Trembling Hands                  9   
...                                                              ...   
42      The Decapitator                                            1   
51      Endbringer                                                 1   
118     Ghost Reaver, Longsword of Magic                           1   
104     Gladiator's Glaive                                         1   
91      Celeste                                                    1   

                                                     Item Price  \
Item ID Item Name                                                 
92      Final Critic                                      $4.61   
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
145     Fiery Glass Crusader                              $4.58   
132     Persuasion                                        $3.22   
108     Extraction, Quickblade Of Trembling Hands         $3.53   
...                                                         ...   
42      The Decapitator                                   $1.75   
51      Endbringer                                        $4.66   
118     Ghost Reaver, Longsword of Magic                  $2.17   
104     Gladiator's Glaive                                $1.93   
91      Celeste                                           $4.17   

                                                     Total Purchase Value  
Item ID Item Name                                                          
92      Final Critic                                               $59.99  
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
145     Fiery Glass Crusader                                       $41.22  
132     Persuasion                                                 $28.99  
108     Extraction, Quickblade Of Trembling Hands                  $31.77  
...                                                                   ...  
42      The Decapitator                                             $1.75  
51      Endbringer                                                  $4.66  
118     Ghost Reaver, Longsword of Magic                            $2.17  
104     Gladiator's Glaive                                          $1.93  
91      Celeste                                                     $4.17  

[179 rows x 3 columns]

In [11]:
    # Most Profitable Items
# make a copy of previous dataframe to new one
profit_df = popular_df

# override total purchase value column with the values (change the data type back to numeric)
profit_df["Total Purchase Value"] = item_df["Price"].sum()

# reorganize so that its in descending order of total purchase value
profit_df = profit_df.sort_values(by=["Total Purchase Value"], ascending=False)

# reformat and display
profit_df["Total Purchase Value"] = profit_df["Total Purchase Value"].map("${:.2f}".format)
profit_df

Purchase Count  \
Item ID Item Name                                                      
92      Final Critic                                              13   
178     Oathbreaker, Last Hope of the Breaking Storm              12   
82      Nirvana                                                    9   
145     Fiery Glass Crusader                                       9   
103     Singed Scalpel                                             8   
...                                                              ...   
28      Flux, Destroyer of Due Diligence                           2   
126     Exiled Mithril Longsword                                   1   
125     Whistling Mithril Warblade                                 2   
104     Gladiator's Glaive                                         1   
42      The Decapitator                                            1   

                                                     Item Price  \
Item ID Item Name                                                 
92      Final Critic                                      $4.61   
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
82      Nirvana                                           $4.90   
145     Fiery Glass Crusader                              $4.58   
103     Singed Scalpel                                    $4.35   
...                                                         ...   
28      Flux, Destroyer of Due Diligence                  $1.06   
126     Exiled Mithril Longsword                          $2.00   
125     Whistling Mithril Warblade                        $1.00   
104     Gladiator's Glaive                                $1.93   
42      The Decapitator                                   $1.75   

                                                     Total Purchase Value  
Item ID Item Name                                                          
92      Final Critic                                               $59.99  
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
82      Nirvana                                                    $44.10  
145     Fiery Glass Crusader                                       $41.22  
103     Singed Scalpel                                             $34.80  
...                                                                   ...  
28      Flux, Destroyer of Due Diligence                            $2.12  
126     Exiled Mithril Longsword                                    $2.00  
125     Whistling Mithril Warblade                                  $2.00  
104     Gladiator's Glaive                                          $1.93  
42      The Decapitator                                             $1.75  

[179 rows x 3 columns]